# 01 · Data Sanity — быстрый рентген данных ⚙️

Цель:
- проверить схему, время, дубликаты/пропуски, кардинальности, холодный старт;
- построить (по необходимости) `session_id`;
- сделать time-safe сплиты с эмбарго;
- сохранить нормализованные parquet + `splits.json` + краткий `report.json`.

**Стоп-условия:** пропуски в ключах, грубые утечки времени, пустой вал/тест.

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, math, glob, shlex, subprocess, warnings, pathlib, time
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

from IPython.display import display, clear_output, HTML
import ipywidgets as W

pd.set_option("display.max_colwidth", 120)
warnings.filterwarnings("ignore")

REPO = Path.cwd()
ART_ROOT = Path("artifacts/recsys/dataio")

def mem_gb(obj=None):
    try:
        import psutil
        if obj is None:
            return psutil.Process().memory_info().rss / (1024**3)
    except Exception:
        pass
    try:
        if hasattr(obj, "memory_usage"):
            return float(obj.memory_usage(deep=True).sum())/(1024**3)
        return float(np.array(obj).nbytes)/(1024**3)
    except Exception:
        return float('nan')

def peek(df, n=5):
    display(df.head(n))
    print(df.shape, "| mem:", f"{mem_gb(df):.3f} GB")

def parse_embargo(s: str) -> pd.Timedelta:
    """Строка вида '2D', '12h', '30m' → Timedelta."""
    if s is None or str(s).strip()=="":
        return pd.Timedelta(0)
    s = str(s).strip().lower()
    if s.endswith("d"):
        return pd.Timedelta(days=float(s[:-1]))
    if s.endswith("h"):
        return pd.Timedelta(hours=float(s[:-1]))
    if s.endswith("m"):
        return pd.Timedelta(minutes=float(s[:-1]))
    # по умолчанию — часы
    try:
        return pd.Timedelta(hours=float(s))
    except:
        return pd.Timedelta(0)

def to_utc(series):
    """Парсинг времени в UTC pandas datetime64[ns, UTC]."""
    s = pd.to_datetime(series, errors="coerce", utc=True)
    return s

def ts_floor_hour(ts: pd.Timestamp) -> pd.Timestamp:
    if ts.tz is None:
        ts = ts.tz_localize("UTC")
    return ts.floor("h")

In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
STATE = dict(
    DATASET_ID="s5e11",
    RAW_INTERACTIONS="data/s5e11/interactions.parquet",
    RAW_ITEMS="data/s5e11/items.parquet",
    SESSION_GAP_MIN=30,
    MAX_SESSION_LEN=200,
    VAL_FRAC=0.1,
    TEST_FRAC=0.1,
    EMBARGO="2D",
)

# Виджеты
w_dataset  = W.Text(STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="280px"))
w_inter    = W.Text(STATE["RAW_INTERACTIONS"], description="interactions:", layout=W.Layout(width="640px"))
w_items    = W.Text(STATE["RAW_ITEMS"], description="items:", layout=W.Layout(width="640px"))

w_gap      = W.IntText(STATE["SESSION_GAP_MIN"], description="session_gap_min:", layout=W.Layout(width="220px"))
w_maxlen   = W.IntText(STATE["MAX_SESSION_LEN"], description="max_session_len:", layout=W.Layout(width="220px"))
w_valfrac  = W.FloatText(STATE["VAL_FRAC"], description="val_frac:", layout=W.Layout(width="180px"))
w_testfrac = W.FloatText(STATE["TEST_FRAC"], description="test_frac:", layout=W.Layout(width="180px"))
w_embargo  = W.Text(STATE["EMBARGO"], description="embargo:", layout=W.Layout(width="180px"))

btn_apply  = W.Button(description="Применить параметры", button_style="success", layout=BTN_LAYOUT)
out_apply  = W.Output()

def on_apply(_):
    with out_apply:
        clear_output()
        STATE["DATASET_ID"]        = w_dataset.value.strip()
        STATE["RAW_INTERACTIONS"]  = w_inter.value.strip()
        STATE["RAW_ITEMS"]         = w_items.value.strip()
        STATE["SESSION_GAP_MIN"]   = int(w_gap.value)
        STATE["MAX_SESSION_LEN"]   = int(w_maxlen.value)
        STATE["VAL_FRAC"]          = float(w_valfrac.value)
        STATE["TEST_FRAC"]         = float(w_testfrac.value)
        STATE["EMBARGO"]           = w_embargo.value.strip()
        print(json.dumps(STATE, ensure_ascii=False, indent=2))
btn_apply.on_click(on_apply)

W.VBox([
    W.HTML("<h3>Параметры</h3>"),
    w_dataset,
    w_inter, 
    w_items,
    W.HBox([w_gap, w_maxlen, w_valfrac, w_testfrac, w_embargo], layout=ROW_LAYOUT),
    btn_apply,
    out_apply
])

In [ ]:
out_load = W.Output()
btn_load = W.Button(description="Загрузить сырьё и показать срезы", button_style="primary", layout=BTN_LAYOUT)

RAW = dict(inter=None, items=None)

def load_raw():
    p_int = Path(STATE["RAW_INTERACTIONS"])
    p_itm = Path(STATE["RAW_ITEMS"])
    if not p_int.exists():
        raise FileNotFoundError(f"Не найден interactions: {p_int}")
    if not p_itm.exists():
        raise FileNotFoundError(f"Не найден items: {p_itm}")
    inter = pd.read_parquet(p_int) if p_int.suffix==".parquet" else pd.read_csv(p_int)
    items = pd.read_parquet(p_itm) if p_itm.suffix==".parquet" else pd.read_csv(p_itm)
    return inter, items

def on_load(_):
    with out_load:
        clear_output()
        inter, items = load_raw()
        RAW["inter"] = inter
        RAW["items"] = items
        print("interactions:")
        peek(inter)
        print("items:")
        peek(items)
        # ключевые столбцы
        need = {"user_id","item_id","ts"}
        missing = need - set(inter.columns)
        if missing:
            raise AssertionError(f"В interactions нет ключевых колонок: {missing}")
        if inter.shape[0]==0 or items.shape[0]==0:
            raise AssertionError("Один из датафреймов пуст")

btn_load.on_click(on_load)
W.VBox([btn_load, out_load])

In [ ]:
out_schema = W.Output()
btn_schema = W.Button(description="Валидировать схему и привести типы", button_style="primary", layout=BTN_LAYOUT)

NORM = dict(inter=None, items=None, notes=dict())

def normalize_schema(inter: pd.DataFrame, items: pd.DataFrame):
    inter = inter.copy()
    items = items.copy()
    notes = {}

    # Обязательные
    for col in ["user_id","item_id"]:
        if col not in inter.columns:
            raise AssertionError(f"[schema] Нет колонки {col} в interactions")
    if "ts" not in inter.columns:
        raise AssertionError("[schema] Нет временной колонки ts в interactions")

    # Каст ключей
    for col in ["user_id","item_id"]:
        if not pd.api.types.is_integer_dtype(inter[col]):
            inter[col] = pd.to_numeric(inter[col], errors="coerce").astype("Int64").astype("int64")
        if not pd.api.types.is_integer_dtype(items[col]) and col in items.columns:
            items[col] = pd.to_numeric(items[col], errors="coerce").astype("Int64").astype("int64")

    # Время
    inter["ts"] = to_utc(inter["ts"])
    if inter["ts"].isna().mean() > 0.01:
        raise AssertionError("[schema] >1% нераспарсенных ts")

    # Каст частых полей
    opt_num = ["price"]
    for c in opt_num:
        if c in inter.columns:
            inter[c] = pd.to_numeric(inter[c], errors="coerce").astype("float32")
        if c in items.columns:
            items[c] = pd.to_numeric(items[c], errors="coerce").astype("float32")
    for c in ["brand","category","event_type","title"]:
        if c in inter.columns:
            inter[c] = inter[c].astype("string")
        if c in items.columns:
            items[c] = items[c].astype("string")

    if "created_ts" in items.columns:
        items["created_ts"] = to_utc(items["created_ts"])

    # Доп. колонки времени
    inter = inter.sort_values(["user_id","ts"]).reset_index(drop=True)
    inter["date"] = inter["ts"].dt.date
    inter["hour"] = inter["ts"].dt.hour.astype("int16")
    inter["dow"]  = inter["ts"].dt.dayofweek.astype("int16")

    # Пропуски в ключах
    if inter[["user_id","item_id","ts"]].isna().any().any():
        raise AssertionError("[schema] Пропуски в ключевых колонках" )

    notes["counts"] = dict(n_events=int(inter.shape[0]), n_users=int(inter["user_id"].nunique()), n_items=int(inter["item_id"].nunique()))
    notes["time_span"] = dict(min=str(inter["ts"].min()), max=str(inter["ts"].max()))
    return inter, items, notes

def on_schema(_):
    with out_schema:
        clear_output()
        if RAW["inter"] is None:
            print("Сперва загрузите данные (кнопка выше).")
            return
        inter, items, notes = normalize_schema(RAW["inter"], RAW["items"])
        NORM["inter"], NORM["items"], NORM["notes"] = inter, items, notes
        print("✓ Схема ок. Срезы:")
        print(json.dumps(notes, ensure_ascii=False, indent=2))
        peek(inter)
        if "created_ts" in items.columns:
            print("items[created_ts] диапазон:", items["created_ts"].min(), "→", items["created_ts"].max())
        peek(items)

btn_schema.on_click(on_schema)
W.VBox([btn_schema, out_schema])

In [ ]:
out_dups = W.Output()
btn_dups = W.Button(description="Проверить и убрать дубликаты", button_style="primary", layout=BTN_LAYOUT)

QC = dict(dup_rate=None, nan_by_col=None)

def drop_dups(inter: pd.DataFrame):
    key_cols = [c for c in ["user_id","item_id","ts","event_type"] if c in inter.columns]
    before = inter.shape[0]
    inter2 = inter.drop_duplicates(subset=key_cols, keep="first")
    after = inter2.shape[0]
    dup_rate = (before - after) / max(1,before)
    nans = inter2.isna().mean().sort_values(ascending=False).head(10).to_dict()
    return inter2, dup_rate, nans

def on_dups(_):
    with out_dups:
        clear_output()
        if NORM["inter"] is None:
            print("Сначала — валидация схемы.")
            return
        inter2, dup_rate, nans = drop_dups(NORM["inter"])
        NORM["inter"] = inter2.reset_index(drop=True)
        QC["dup_rate"] = dup_rate
        QC["nan_by_col"] = nans
        print(f"dup_rate: {dup_rate:.4%}")
        print("NaN top-10:", json.dumps(nans, ensure_ascii=False, indent=2))
        peek(inter2)

btn_dups.on_click(on_dups)
W.VBox([btn_dups, out_dups])

In [ ]:
out_sess = W.Output()
btn_sess = W.Button(description="Построить session_id", button_style="primary", layout=BTN_LAYOUT)

def build_sessions(inter: pd.DataFrame, gap_min=30, max_len=200):
    inter = inter.sort_values(["user_id","ts"]).reset_index(drop=True)
    # разрывы по пользователю
    gid = (inter["user_id"] != inter["user_id"].shift(1)).astype(int)
    dt = inter["ts"].view("int64").diff().fillna(10**18)  # наносекунды
    gap_ns = int(gap_min*60*1e9)
    cut = (dt > gap_ns).astype(int)
    # новая сессия если: новый user или gap или достигнут лимит длины
    # накапливаем длину текущей сессии
    sess_len = np.zeros(len(inter), dtype=np.int32)
    sess_len[0] = 1
    for i in range(1, len(inter)):
        if gid[i]==1 or cut[i]==1 or sess_len[i-1] >= max_len:
            sess_len[i] = 1
        else:
            sess_len[i] = sess_len[i-1] + 1
    # формируем session_id как кумулятивную сумму «стартов»
    start = ((sess_len==1).astype(int)).cumsum()
    # приклеим user, чтобы уникальность была глобальной
    inter["session_id"] = (inter["user_id"].astype("int64")<<32) + start.astype("int64")
    stats = {
        "n_sessions": int(inter["session_id"].nunique()),
        "len_p50": int(inter.groupby("session_id").size().median()),
        "len_p90": int(inter.groupby("session_id").size().quantile(0.9))
    }
    return inter, stats

def on_sess(_):
    with out_sess:
        clear_output()
        if NORM["inter"] is None:
            print("Сначала — валидация схемы.")
            return
        inter2, stats = build_sessions(NORM["inter"], gap_min=STATE["SESSION_GAP_MIN"], max_len=STATE["MAX_SESSION_LEN"])
        NORM["inter"] = inter2
        NORM["notes"]["sessions"] = stats
        print("✓ sessions построены:", stats)
        peek(inter2[["user_id","ts","session_id"]])

btn_sess.on_click(on_sess)
W.VBox([btn_sess, out_sess])

In [ ]:
out_split = W.Output()
btn_split = W.Button(description="Построить time-splits + эмбарго", button_style="primary", layout=BTN_LAYOUT)

SPLITS = dict(bounds=None, sizes=None, embargo=STATE["EMBARGO"])

def build_time_splits(inter: pd.DataFrame, val_frac=0.1, test_frac=0.1):
    inter = inter.sort_values("ts")
    t_min, t_max = inter["ts"].min(), inter["ts"].max()
    if not (0 < val_frac < 0.5 and 0 < test_frac < 0.5 and (val_frac+test_frac)<0.8):
        raise AssertionError("Некорректные доли val/test")
    n = len(inter)
    q_train_end = 1 - (val_frac + test_frac)
    q_val_end   = 1 - test_frac
    t_train_end = inter["ts"].quantile(q_train_end)
    t_val_end   = inter["ts"].quantile(q_val_end)

    # округлим до часа
    t_train_end = ts_floor_hour(t_train_end)
    t_val_end   = ts_floor_hour(t_val_end)

    train = inter[inter["ts"] <= t_train_end]
    val   = inter[(inter["ts"] > t_train_end) & (inter["ts"] <= t_val_end)]
    test  = inter[inter["ts"] > t_val_end]

    sizes = dict(train=int(train.shape[0]), val=int(val.shape[0]), test=int(test.shape[0]))
    bounds = dict(t_min=str(t_min), train_end=str(t_train_end), val_end=str(t_val_end), t_max=str(t_max))
    return bounds, sizes

def check_embargo(inter: pd.DataFrame, bounds: dict, embargo_str: str):
    emb = parse_embargo(embargo_str)
    if emb == pd.Timedelta(0):
        return dict(ok=True, msg="embargo=0")
    t_train_end = pd.to_datetime(bounds["train_end"])
    t_val_start = t_train_end + pd.Timedelta(0)
    # запретить события ближе эмбарго к границе
    train_edge = inter[(inter["ts"] > t_train_end - emb) & (inter["ts"] <= t_train_end)]
    val_edge   = inter[(inter["ts"] > t_train_end) & (inter["ts"] <= t_train_end + emb)]
    ok = len(val_edge)==0  # строгий вариант: вал начинается после окна эмбарго
    msg = f"near-train-end: train_edge={len(train_edge)}, val_edge={len(val_edge)}, embargo={emb}"
    return dict(ok=ok, msg=msg)

def on_split(_):
    with out_split:
        clear_output()
        if NORM["inter"] is None:
            print("Сначала — валидация схемы.")
            return
        bounds, sizes = build_time_splits(NORM["inter"], STATE["VAL_FRAC"], STATE["TEST_FRAC"])
        SPLITS["bounds"], SPLITS["sizes"], SPLITS["embargo"] = bounds, sizes, STATE["EMBARGO"]
        print("bounds:", json.dumps(bounds, indent=2))
        print("sizes:", json.dumps(sizes, indent=2))
        embargo_check = check_embargo(NORM["inter"], bounds, STATE["EMBARGO"])
        print("embargo_check:", embargo_check)
        if sizes["val"]==0 or sizes["test"]==0:
            raise AssertionError("Один из сплитов пуст (val/test)")
        if not embargo_check["ok"]:
            print("⚠️ embargo предупреждение:", embargo_check["msg"])

btn_split.on_click(on_split)
W.VBox([btn_split, out_split])

In [ ]:
out_cs = W.Output()
btn_cs = W.Button(description="Посчитать холодный старт/coverage", button_style="primary", layout=BTN_LAYOUT)

COVER = dict(cold_items=None, cold_users=None, overlap=None)

def cold_start_stats(inter: pd.DataFrame, bounds: dict):
    t_train_end = pd.to_datetime(bounds["train_end"])
    t_val_end   = pd.to_datetime(bounds["val_end"])
    tr = inter[inter["ts"] <= t_train_end]
    va = inter[(inter["ts"] > t_train_end) & (inter["ts"] <= t_val_end)]
    te = inter[inter["ts"] > t_val_end]

    tr_items = set(tr["item_id"].unique()); va_items = set(va["item_id"].unique()); te_items = set(te["item_id"].unique())
    tr_users = set(tr["user_id"].unique()); va_users = set(va["user_id"].unique()); te_users = set(te["user_id"].unique())

    cold_items_val = len(va_items - tr_items) / max(1,len(va_items))
    cold_items_test= len(te_items - tr_items) / max(1,len(te_items))
    cold_users_val = len(va_users - tr_users) / max(1,len(va_users))
    cold_users_test= len(te_users - tr_users) / max(1,len(te_users))

    overlap = dict(
        item_val_vs_train = len(va_items & tr_items) / max(1,len(va_items)),
        item_test_vs_train= len(te_items & tr_items) / max(1,len(te_items)),
    )
    return dict(
        cold_items=dict(val=cold_items_val, test=cold_items_test),
        cold_users=dict(val=cold_users_val, test=cold_users_test),
        overlap=overlap
    )

def on_cs(_):
    with out_cs:
        clear_output()
        if NORM["inter"] is None or SPLITS["bounds"] is None:
            print("Нужны нормализованные данные и сплиты.")
            return
        stats = cold_start_stats(NORM["inter"], SPLITS["bounds"])
        COVER.update(stats)
        print(json.dumps(stats, indent=2))

btn_cs.on_click(on_cs)
W.VBox([btn_cs, out_cs])

In [ ]:
out_lbl = W.Output()
btn_lbl = W.Button(description="Проверить лейблы / баланс", button_style="primary", layout=BTN_LAYOUT)

LABELS = dict(pos_rate=None, drift=None)

def labels_stats(inter: pd.DataFrame):
    if "label" not in inter.columns:
        return dict(has_label=False)
    inter = inter.copy()
    inter["label"] = pd.to_numeric(inter["label"], errors="coerce").fillna(0).astype(int)
    pos_rate = float(inter["label"].mean())
    drift = inter.set_index("ts")["label"].resample("D").mean().dropna().tail(30).to_dict()
    return dict(has_label=True, pos_rate=pos_rate, drift_last30=drift)

def on_lbl(_):
    with out_lbl:
        clear_output()
        if NORM["inter"] is None:
            print("Сначала — валидация схемы.")
            return
        stats = labels_stats(NORM["inter"])
        LABELS.update(stats)
        print(json.dumps(stats, indent=2))

btn_lbl.on_click(on_lbl)
W.VBox([btn_lbl, out_lbl])

In [ ]:
out_leak = W.Output()
btn_leak = W.Button(description="Анти-утечки: быстрые проверки", button_style="warning", layout=BTN_LAYOUT)

LEAK = dict(items_future=False, msg_items="", ok=True)

def check_item_created_vs_events(inter: pd.DataFrame, items: pd.DataFrame):
    if "created_ts" not in items.columns:
        return dict(ok=True, msg="no created_ts in items")
    # join по item_id (пример: случайная 100k для скорости)
    sample = inter.sample(min(len(inter), 100_000), random_state=1)
    df = sample[["item_id","ts"]].merge(items[["item_id","created_ts"]], on="item_id", how="left")
    bad = df[df["created_ts"].notna() & (df["created_ts"] > df["ts"])].shape[0]
    return dict(ok=(bad==0), msg=f"items created_ts > ts events: {bad} / {len(df)} (sample)")

def check_embargo_strict(inter: pd.DataFrame, bounds: dict, embargo_str: str):
    emb = parse_embargo(embargo_str)
    if emb == pd.Timedelta(0):
        return dict(ok=True, msg="embargo=0")
    t_train_end = pd.to_datetime(bounds["train_end"])
    val_st = t_train_end + pd.Timedelta(0)
    # строгий вариант: минимальный ts в val должен быть >= train_end + emb
    min_val = inter[inter["ts"] > t_train_end]["ts"].min()
    ok = pd.isna(min_val) or (min_val >= t_train_end + emb)
    return dict(ok=ok, msg=f"min_val={min_val}, needs ≥ {t_train_end+emb}")

def on_leak(_):
    with out_leak:
        clear_output()
        if NORM["inter"] is None:
            print("Сначала — валидация схемы.")
            return
        res1 = check_item_created_vs_events(NORM["inter"], NORM["items"])
        print("items future check:", res1)
        res2 = dict(ok=True, msg="skip (нет сплитов)")
        if SPLITS["bounds"] is not None:
            res2 = check_embargo_strict(NORM["inter"], SPLITS["bounds"], STATE["EMBARGO"])
            print("embargo strict check:", res2)
        ok = res1["ok"] and res2["ok"]
        LEAK["ok"] = ok
        LEAK["items_future"] = not res1["ok"]
        LEAK["msg_items"] = res1["msg"]
        print("→ LEAK OK:", ok)

btn_leak.on_click(on_leak)
W.VBox([btn_leak, out_leak])

In [ ]:
out_save = W.Output()
btn_save = W.Button(description="Сохранить parquet + splits + report", button_style="success", layout=BTN_LAYOUT)

REPORT = dict()

def save_artifacts(inter: pd.DataFrame, items: pd.DataFrame, dataset_id: str, splits: dict, qc: dict, cover: dict, labels: dict, leak: dict):
    out_dir = ART_ROOT / dataset_id
    out_dir.mkdir(parents=True, exist_ok=True)
    # interactions_norm
    inter2 = inter.copy()
    # упорядочим столбцы
    base_cols = [c for c in ["user_id","item_id","ts","session_id","event_type","price","brand","category","label"] if c in inter2.columns]
    other_cols= [c for c in inter2.columns if c not in base_cols]
    inter2 = inter2[base_cols + other_cols]
    inter2.to_parquet(out_dir/"interactions_norm.parquet", index=False)
    # items_norm
    items2 = items.copy()
    items2.to_parquet(out_dir/"items_norm.parquet", index=False)
    # interactions_with_split — по желанию (здесь не пишем, сплиты в json)
    # splits.json
    splits_out = dict(bounds=splits.get("bounds"), sizes=splits.get("sizes"), embargo=splits.get("embargo"))
    (out_dir/"splits.json").write_text(json.dumps(splits_out, ensure_ascii=False, indent=2))

    # report.json
    rep = dict(
        counts=NORM["notes"].get("counts",{}),
        time_span=NORM["notes"].get("time_span",{}),
        sessions=NORM["notes"].get("sessions",{}),
        dup_rate=qc.get("dup_rate"),
        nan_by_col=qc.get("nan_by_col"),
        splits=splits_out,
        cold_start=cover,
        labels=labels,
        leak_checks=leak,
        saved_at=datetime.utcnow().isoformat()+"Z"
    )
    (out_dir/"report.json").write_text(json.dumps(rep, ensure_ascii=False, indent=2))
    return out_dir, rep

def on_save(_):
    with out_save:
        clear_output()
        if NORM["inter"] is None or SPLITS["bounds"] is None:
            print("Нужны нормализованные данные и сплиты.")
            return
        out_dir, rep = save_artifacts(NORM["inter"], NORM["items"], STATE["DATASET_ID"], SPLITS, QC, COVER, LABELS, LEAK)
        REPORT.update(rep)
        print("✓ Сохранено в:", out_dir)
        print(json.dumps(rep, ensure_ascii=False, indent=2))

btn_save.on_click(on_save)
W.VBox([btn_save, out_save])

In [ ]:
out_gate = W.Output()
btn_gate = W.Button(description="Показать PASS/FAIL и рекомендации", button_style="info", layout=BTN_LAYOUT)

def pass_fail():
    fails = []
    warns = []
    # Критичные
    if QC.get("dup_rate", 0) > 0.01:
        warns.append(f"Высокие дубликаты: {QC['dup_rate']:.2%} (норм <0.5%)")
    if NORM["inter"][["user_id","item_id","ts"]].isna().any().any():
        fails.append("NaN в ключевых колонках")
    if SPLITS.get("sizes",{}).get("val",0)==0 or SPLITS.get("sizes",{}).get("test",0)==0:
        fails.append("Пустой val/test")
    if not LEAK.get("ok", True):
        fails.append("Сработали анти-утечки")
    # Холодный старт
    cs = COVER.get("cold_items",{})
    if cs:
        if cs.get("test",0) > 0.3:
            warns.append(f"Сильный холодный старт по item в test: {cs['test']:.1%} → усиливай retrieval (контент/симилярити)")
    return fails, warns

def on_gate(_):
    with out_gate:
        clear_output()
        if not REPORT:
            print("Сначала сохраните артефакты (будет report.json).")
            return
        fails, warns = pass_fail()
        html = "<h3>Итог</h3>"
        if fails:
            html += f"<p style='color:#c62828'><b>FAILS:</b><br>• " + "<br>• ".join(fails) + "</p>"
        else:
            html += "<p style='color:#2e7d32'><b>PASS:</b> критичных проблем не обнаружено</p>"
        if warns:
            html += f"<p style='color:#f57c00'><b>WARNS:</b><br>• " + "<br>• ".join(warns) + "</p>"
        html += "<hr><p>Дальше: открой 02_candidates_lab.ipynb (или запусти retrieval из runbook).</p>"
        display(HTML(html))

btn_gate.on_click(on_gate)
W.VBox([btn_gate, out_gate])

In [ ]:
btn_all = W.Button(description="🚀 Быстрый прогон: load → schema → dups → sessions → splits → labels → CS → save → gate", button_style="success", layout=BTN_LAYOUT)
out_all = W.Output()

def on_all(_):
    with out_all:
        clear_output()
        try:
            print("1) load")
            inter, items = load_raw()
            RAW["inter"], RAW["items"] = inter, items

            print("2) schema")
            inter, items, notes = normalize_schema(inter, items)
            NORM["inter"], NORM["items"], NORM["notes"] = inter, items, notes

            print("3) dups")
            inter2, dup_rate, nans = drop_dups(inter)
            NORM["inter"] = inter2
            QC["dup_rate"], QC["nan_by_col"] = dup_rate, nans

            print("4) sessions")
            inter3, stats = build_sessions(NORM["inter"], STATE["SESSION_GAP_MIN"], STATE["MAX_SESSION_LEN"])
            NORM["inter"] = inter3
            NORM["notes"]["sessions"] = stats

            print("5) splits")
            bounds, sizes = build_time_splits(NORM["inter"], STATE["VAL_FRAC"], STATE["TEST_FRAC"])
            SPLITS["bounds"], SPLITS["sizes"], SPLITS["embargo"] = bounds, sizes, STATE["EMBARGO"]

            print("6) labels")
            LABELS.update(labels_stats(NORM["inter"]))

            print("7) cold start")
            COVER.update(cold_start_stats(NORM["inter"], SPLITS["bounds"]))

            print("8) leaks")
            res1 = check_item_created_vs_events(NORM["inter"], NORM["items"])
            res2 = check_embargo_strict(NORM["inter"], SPLITS["bounds"], STATE["EMBARGO"])
            LEAK["ok"] = res1["ok"] and res2["ok"]
            LEAK["items_future"] = not res1["ok"]
            LEAK["msg_items"] = res1["msg"]

            print("9) save")
            out_dir, rep = save_artifacts(NORM["inter"], NORM["items"], STATE["DATASET_ID"], SPLITS, QC, COVER, LABELS, LEAK)
            REPORT.update(rep)
            print("✓ done →", out_dir)
        except Exception as e:
            print("ОШИБКА:", e)
            raise

btn_all.on_click(on_all)
W.VBox([btn_all, out_all])

In [ ]:
out_list = W.Output()
btn_list = W.Button(description="Показать сохранённые файлы", button_style="", layout=BTN_LAYOUT)

def on_list(_):
    with out_list:
        clear_output()
        d = ART_ROOT / STATE["DATASET_ID"]
        if not d.exists():
            print("Каталог еще не создан:", d)
            return
        files = sorted(d.rglob("*"))
        df = pd.DataFrame([{ "path":str(p), "size_MB": round(p.stat().st_size/1024/1024,3)} for p in files if p.is_file()])
        display(df)
btn_list.on_click(on_list)
W.VBox([btn_list, out_list])